In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [81]:
fraud_data = pd.read_csv('Fraud_Data.csv')
ip_address = pd.read_csv('IpAddress_to_Country.csv')

In [86]:
# function that takes an IP address as argument and 
# returns country associated based on IP_table
def ip_to_country(ip=None):
    try :
        return ip_address.country[(ip_address.lower_bound_ip_address <= ip) 
                             & (ip_address.upper_bound_ip_address >= ip)].iloc[0]
    except IndexError :
        return "Unknown"     
# To affect a country to each IP :
fraud_data["ip_country"] = fraud_data.ip_address.apply(ip_to_country)

In [83]:
## ajout d'un diff_time en nombre de secondes
fraud_data.signup_time = pd.to_datetime(fraud_data.signup_time)
fraud_data.purchase_time = pd.to_datetime(fraud_data.purchase_time)
fraud_data['diff_time3']=fraud_data.purchase_time-fraud_data.signup_time

In [84]:
fraud_data['diff_time3'] =fraud_data['diff_time3'].dt.total_seconds()

In [147]:
#ajout d'un diff_time en nombre d'heures
fraud_data['diff_time'] =fraud_data.purchase_time-fraud_data.signup_time
fraud_data['diff_time'] =fraud_data['diff_time3'].dt.total_seconds()

# ancienne méthode pas glop
#for i in range(len(fraud_data)):
    fraud_data['diff_time'][i]=24*fraud_data['diff_time'][i].days+round(fraud_data['diff_time'][i].seconds/3600)

C:\Users\djock\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
fraud_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         151112 non-null  int64         
 1   signup_time     151112 non-null  datetime64[ns]
 2   purchase_time   151112 non-null  datetime64[ns]
 3   purchase_value  151112 non-null  int64         
 4   device_id       151112 non-null  object        
 5   source          151112 non-null  object        
 6   browser         151112 non-null  object        
 7   sex             151112 non-null  object        
 8   age             151112 non-null  int64         
 9   ip_address      151112 non-null  float64       
 10  class           151112 non-null  int64         
 11  diff_time3      151112 non-null  float64       
 12  ip_country      151112 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(4), object(5)
memory usage: 15.0+ MB


In [ ]:
ip_address.head()

In [88]:
fraud_data = fraud_data.drop(["signup_time", "purchase_time", "ip_address","device_id","user_id"], axis = 1)
fraud_data = fraud_data.dropna()
fraud_data = pd.get_dummies(fraud_data)
X = fraud_data.drop("class", axis = 1)
y = fraud_data["class"]

In [89]:
X.head()

,purchase_value,age,diff_time3,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,...,ip_country_Unknown,ip_country_Uruguay,ip_country_Uzbekistan,ip_country_Vanuatu,ip_country_Venezuela,ip_country_Viet Nam,ip_country_Virgin Islands (U.S.),ip_country_Yemen,ip_country_Zambia,ip_country_Zimbabwe
0,34,39,4506682.0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16,53,17944.0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,53,1.0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,44,41,492085.0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,39,45,4361461.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0, stratify = y)

In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [92]:
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

models = [LogisticRegression(), DecisionTreeClassifier(),RandomForestClassifier()]

In [75]:
for model in models:
    %time model.fit(X_train_scaled, y_train)

Wall time: 1.39 s
Wall time: 6.92 s
Wall time: 43.7 s


In [76]:
list_pred = []
#y_pred_tmp=np.zeros(len(y_test))
for model in models:
    y_pred_tmp = model.predict(X_test_scaled)
    list_pred.append(y_pred_tmp)

In [77]:
for i in range(len(list_pred)):
    print(confusion_matrix(list_pred[i],y_test))
    print("accuracy score:",accuracy_score(list_pred[i],y_test))
    print("f1 score: ",f1_score(list_pred[i],y_test))

[[27385  2799]
 [    8    31]]
accuracy score: 0.9071237137279555
f1 score:  0.021610317183687697
[[25829  1196]
 [ 1564  1634]]
accuracy score: 0.9086788207656421
f1 score:  0.5421366954213669
[[27374  1273]
 [   19  1557]]
accuracy score: 0.9572511001555107
f1 score:  0.7067635043123014


In [93]:
from sklearn.ensemble import VotingClassifier
clf = VotingClassifier(
    estimators = [ 
                  ("logreg", LogisticRegression()), 
                  ("tree", DecisionTreeClassifier()),
        ("RdmFo",RandomForestClassifier())
                  ]
)

In [94]:
%time clf.fit(X_train_scaled, y_train)

Wall time: 53.6 s


VotingClassifier(estimators=[('logreg',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='...
                                        

In [95]:
clf.score(X_test_scaled, y_test)

0.957449624458194

In [96]:
y_pred_clf = clf.predict(X_test_scaled)

In [97]:
confusion_matrix(y_pred_clf,y_test)

array([[27382,  1275],
       [   11,  1555]], dtype=int64)

In [98]:
f1_score(y_pred_clf,y_test)

0.7074613284804369

In [99]:
from sklearn.ensemble import StackingClassifier
sclf = StackingClassifier(
    estimators = [
                  ("logreg", LogisticRegression()), 
                  ("tree", DecisionTreeClassifier()),
        ("RdmFo",RandomForestClassifier())
                  ],
    final_estimator=LogisticRegression(),n_jobs=-1,verbose =2
)

In [100]:
%time sclf.fit(X_train_scaled, y_train)

Wall time: 1min 55s


StackingClassifier(cv=None,
                   estimators=[('logreg',
                                LogisticRegression(C=1.0, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=None, penalty='l2',
                                                   random_state=None,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('tree',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                          

In [101]:
y_pred_sclf = sclf.predict(X_test_scaled)

In [102]:
print(confusion_matrix(y_pred_sclf,y_test))
print("accuracy score: ",accuracy_score(y_pred_sclf,y_test))
print("f1 score: ",f1_score(y_pred_sclf,y_test))

[[27374  1274]
 [   19  1556]]
accuracy score:  0.9572180127717301
f1 score:  0.7064699205448354


In [ ]:
!pip install xgboost

In [103]:
from xgboost import XGBClassifier
xgbC = XGBClassifier(n_jobs=-1,verbose=2,n_estimator=40,learning_rate=0.5)

In [104]:
%time xgbC.fit(X_train_scaled, y_train)

Wall time: 15.5 s


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.5, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimator=40, n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbose=2, verbosity=None)

In [105]:
y_pred_xgb = xgbC.predict(X_test_scaled)
print(confusion_matrix(y_pred_xgb,y_test))
print("accuracy score: ",accuracy_score(y_pred_xgb,y_test))
print("f1 score: ",f1_score(y_pred_xgb,y_test))

[[27384  1276]
 [    9  1554]]
accuracy score:  0.9574827118419746
f1 score:  0.7074891873435011


In [60]:
from sklearn.model_selection import GridSearchCV
classifier = XGBClassifier()
parameters = {
    'n_estimators':[20,25,30,35,40,45],
              "learning_rate":[0.15,0.2,0.25,0.3,0.35,0.4,0.45]
}

In [61]:
classifier_xgb_best = GridSearchCV(classifier, parameters,n_jobs=-1, verbose=2)

In [62]:
classifier_xgb_best.fit(X_train_scaled,y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   51.2s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:  1.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_es...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,

In [63]:
classifier_xgb_best.best_params_

{'learning_rate': 0.15, 'n_estimators': 20}

In [64]:
y_pred_bestxgb = classifier_xgb_best.predict(X_test_scaled)

In [65]:
print(confusion_matrix(y_pred_bestxgb,y_test))
print("accuracy score: ",accuracy_score(y_pred_bestxgb,y_test))
print("f1 score: ",f1_score(y_pred_bestxgb,y_test))

[[27393  1276]
 [    0  1554]]
accuracy score:  0.9577804982959998
f1 score:  0.7089416058394161


In [39]:
y_test.value_counts()

0    27393
1     2830
Name: class, dtype: int64

array([0, 1], dtype=int64)